CNN LSTM 

Data Set 14
Using data zero data zero padding


In [20]:
%load_ext tensorboard
import tensorflow as tf
import datetime
import keras

import numpy as np
import pandas as pd
from keras.models import Sequential, Model, load_model
from keras.layers import Input ,Dense, Dropout, Activation, LSTM, Conv1D
from keras.layers import Lambda, Convolution1D, MaxPooling1D, Flatten, Reshape, BatchNormalization
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.metrics import categorical_crossentropy, binary_crossentropy

#from spp.SpatialPyramidPooling import SpatialPyramidPooling

#For data saving
import pickle
import random
#other imports
import gzip
import glob
import os
import keras.backend as K
import os
import time 
#cwd = os.path.dirname(os.path.realpath("SURF_001_TwoClass13.ipynb"))

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [21]:
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

In [22]:
dense_layer = 1
layer_size = 64 
conv_layer = 2
seq_length = 1000
base_pair = 4
num_strides = 1
pool_size = 4

epochs = 100
batch_size = 1
val_batch_size = 40

In [23]:
with open('../partition_train_val_a.dat', 'rb') as f:
    partition = pickle.load(f)
train_samples = partition['train']
val_samples = partition['validation']
random.shuffle(train_samples)
random.shuffle(val_samples)
labels = partition['labels']
partition['train'] = train_samples
partition['validation'] = val_samples

print(len(train_samples))
for ID in train_samples:
    if labels[ID][0][0] == 0:
        train_samples.remove(ID)
print(len(train_samples))
print(labels[train_samples[0]])

print(len(val_samples))
for ID in val_samples:
    if labels[ID][0][0] == 0:
        val_samples.remove(ID)
print(len(val_samples))
print(labels[val_samples[0]])


54082
35283
[[1 0]]
13520
8778
[[1 0]]


In [24]:
def remove_zero_category():
    print(len(train_samples))
    for ID in train_samples:
        if labels[ID][0][0] == 0:
            train_samples.remove(ID)
    print(len(train_samples))
    print(labels[train_samples[0]])

    print(len(val_samples))
    for ID in val_samples:
        if labels[ID][0][0] == 0:
            val_samples.remove(ID)
    print(len(val_samples))
    print(labels[val_samples[0]])
    return (train_samples, val_samples)

In [25]:
train_samples, val_samples = remove_zero_category()

35283
27479
[[1 0]]
8778
6760
[[1 0]]


In [28]:
 def train_CNN(train_samples, val_samples, labels):
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('loss')>0.99):
                print("\nReached 99% accuracy so cancelling training!")
                self.model.stop_training = True
    callback = myCallback()
    
    earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False)
    
    #Model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(2, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    #Data
    count_input_train = 0
    count_input_validation = 0
    while count_input_train < len(train_samples):
        Y_train = np.zeros((1, 2), dtype=np.int8)
        X_train = np.zeros((1, 1000, 4), dtype=np.int8)
        Y_val = np.zeros((1, 2), dtype=np.int8)
        X_val = np.zeros((1, 1000, 4), dtype=np.int8)
        if count_input_train < (len(train_samples) -1):
            for i in range(0, 1):
                a = train_samples[count_input_train]
                X_train[i, :, :] = np.load('../model_11_data_4/' + train_samples[count_input_train] + '.npy') 
                Y_train[i, :] = labels[a] 
                count_input_train += 1
        else:
            count_input_train += 1000
        if count_input_validation < (len(val_samples) - 1):
            for i in range(0, 1):
                a = val_samples[count_input_validation]
                X_val[i, :, :] = np.load('../model_11_data_4/' + val_samples[count_input_validation] + '.npy') 
                Y_val[i, :] = labels[a] 
                count_input_validation += 1
        # Fitting Model
        history = model.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=epochs, verbose=2,
                            callbacks=[earlystop], validation_data=(X_val, Y_val), 
                            validation_batch_size=val_batch_size
                           )
    return history.epoch, model


In [ ]:
history, model = train_CNN(train_samples, val_samples, partition['labels'])

Epoch 1/100
1/1 - 1s - loss: 0.7131 - accuracy: 0.0000e+00 - val_loss: 0.1563 - val_accuracy: 1.0000
Epoch 2/100
1/1 - 0s - loss: 1.3410e-04 - accuracy: 1.0000 - val_loss: 0.0485 - val_accuracy: 1.0000
Epoch 3/100
1/1 - 0s - loss: 1.7881e-06 - accuracy: 1.0000 - val_loss: 0.0188 - val_accuracy: 1.0000
Epoch 4/100
1/1 - 0s - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 5/100
1/1 - 0s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0043 - val_accuracy: 1.0000
Epoch 6/100
1/1 - 0s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 7/100
1/1 - 0s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 8/100
1/1 - 0s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 8.9581e-04 - val_accuracy: 1.0000
Epoch 9/100
1/1 - 0s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.9218e-04 - val_accuracy: 1.0000
Epoch 10/100
1/1 - 0s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 4.0761e-04 

In [ ]:
#Testing which model I'll pick


In [8]:
with open('../partition_train_val_a.dat', 'rb') as f:
    partition = pickle.load(f)
train_samples_IDs = partition['train']
val_samples_IDs = partition['validation']
IDs = partition['labels']

In [9]:
#model = load_model('../Models_H5/011_{}-conv-{}-nodes-{}-dense-{}CNNLSTM'.format(1, 128, 1, 1601188309))

In [7]:
predicted = []
Y_val_expected = []
count_input_val = 0
num_val_seq = len(val_samples)
done = False
while count_input_val < num_val_seq and not(done):
    X_val = np.zeros((200, 1000, 4), dtype=np.int8)
    if count_input_val < (num_val_seq - 200):
        for i in range(0, 200):
            a = val_samples[count_input_val]
            X_val[i, :, :] = np.load('../model_11_data_4/' + val_samples[count_input_val] + '.npy') 
            count_input_val += 1
        predict = (model.predict_classes(X_val))
        for label in predict:
            predicted.append(label)
    else:
        done = True
    if count_input_val == 16200:
        done = True
    print(count_input_val)
print(len(predicted))
print(predicted[0])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8600
8600
0


In [8]:
expected = []

for label in val_samples:
    expected.append(labels[label][0][0])
print(expected[0])
print(len(expected))


1
8778


In [9]:
# Example of a confusion matrix in Python
from sklearn.metrics import confusion_matrix
 
results = confusion_matrix(expected[0:13400], predicted)
print(results)
# Printing the precision and recall, among other metrics
#print(metrics.classification_report(y_act, y_pred, labels=["a", 
#"b"]))

ValueError: Found input variables with inconsistent numbers of samples: [8778, 8600]

In [ ]:
import seaborn as sns

sns.heatmap(results/np.sum(results), annot=True, 
            fmt='.2%', cmap='Blues')
print(model.summary)

In [31]:
# Testing Final Model

In [ ]:
with open('../partition_test_a.dat', 'rb') as f:
    partition = pickle.load(f)
test_samples_IDs = partition['test']

with open('../test_labels.dat', 'rb') as f:
    test_labels = pickle.load(f)
print(len(test_labels))


In [55]:
model = load_model('../Models_H5/SURF_Paper_9.26.20/011_{}-conv-{}-nodes-{}-dense-{}CNNLSTM-9-26-20'.format(1, 64, 0, 1601093561))

In [ ]:
predicted = []
count_input_test = 0
num_test_seq = len(partition['test'])
done = False
while count_input_test < num_test_seq and not(done):
    X_test = np.zeros((200, 1000, 4), dtype=np.int8)
    if count_input_test < (num_test_seq - 200):
        for i in range(0, 200):
            a = partition['test'][count_input_test]
            X_test[i, :, :] = np.load('../DataSet_14_Test_2/' + partition['test'][count_input_test] + '.npy') 
            count_input_test += 1
        predict = (model.predict_classes(X_test))
        for label in predict:
            predicted.append(label)
    else:
        done = True
    if count_input_test == 16200:
        done = True
    print(count_input_test)
print(len(predicted))
print(len(test_labels))
print(predicted[0])

In [ ]:
expected = []
for label in test_labels[0:16200]:
    expected.append(label[0][0])
print(expected[0])


In [ ]:
# Example of a confusion matrix in Python
from sklearn.metrics import confusion_matrix
 
results = confusion_matrix(expected, predicted)
print(results)
# Printing the precision and recall, among other metrics
#print(metrics.classification_report(y_act, y_pred, labels=["a", 
#"b"]))

In [ ]:
sns.heatmap(results/np.sum(results), annot=True, 
            fmt='.2%', cmap='Blues')
print(model.summary())

In [ ]:
import tensorflow as tf
print(tf.__version__)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss')<0.4):
      print("\nReached 60% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])
